In [1]:
import argparse
import cv2
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from eval import ToTensor, Normalize
from model import EventDetector
import numpy as np
import tqdm
import torch.nn.functional as F
path = '/home/data/d.kashin/temp/train_dataset'

event_names = {
    0: 'Address',
    1: 'Toe-up',
    2: 'Mid-backswing (arm parallel)',
    3: 'Top',
    4: 'Mid-downswing (arm parallel)',
    5: 'Impact',
    6: 'Mid-follow-through (shaft parallel)',
    7: 'Finish'
}


class SampleVideo(Dataset):
    def __init__(self, path, input_size=160, transform=None):
        self.path = path
        self.input_size = input_size
        self.transform = transform

    def __len__(self):
        return 1

    def __getitem__(self, idx):
        cap = cv2.VideoCapture(self.path)
        frame_size = [cap.get(cv2.CAP_PROP_FRAME_HEIGHT), cap.get(cv2.CAP_PROP_FRAME_WIDTH)]
        ratio = self.input_size / max(frame_size)
        new_size = tuple([int(x * ratio) for x in frame_size])
        #print(frame_size, ratio, new_size)

        delta_w = self.input_size - new_size[1]
        delta_h = self.input_size - new_size[0]
        top, bottom = delta_h // 2, delta_h - (delta_h // 2)
        left, right = delta_w // 2, delta_w - (delta_w // 2)
        #print( top, bottom, left, right)

        # preprocess and return frames
        images = []
        for pos in range(int(cap.get(cv2.CAP_PROP_FRAME_COUNT))):
            ret, img = cap.read()
            if ret:
                resized = cv2.resize(img, (new_size[1], new_size[0]))
                b_img = cv2.copyMakeBorder(resized, top, bottom, left, right, cv2.BORDER_CONSTANT,
                                           value=[0.406 * 255, 0.456 * 255, 0.485 * 255])  # ImageNet means (BGR)

                b_img_rgb = cv2.cvtColor(resized, cv2.COLOR_BGR2RGB)
                images.append(b_img_rgb)
        cap.release()
        labels = np.zeros(len(images)) # only for compatibility with transforms
        sample = {'images': np.asarray(images), 'labels': np.asarray(labels)}
        if self.transform:
            sample = self.transform(sample)
        return sample




In [4]:
model = EventDetector(pretrain=False,
                      width_mult=1.,
                      lstm_layers=1,
                      lstm_hidden=256,
                      bidirectional=True,
                      dropout=False)

try:
    save_dict = torch.load('models/swingnet_1800.pth.tar')
except:
    print("Model weights not found. Download model weights and place in 'models' folder. See README for instructions")

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
model.load_state_dict(save_dict['model_state_dict'])
model.to(device)
model.eval()
print("Loaded model weights")

Using device: cuda
Loaded model weights


In [5]:
seq_length = 64

In [6]:
import pandas as pd
import os

In [7]:
test_path = 'files_test'
names_files = os.listdir(test_path)
files_to_score = [os.path.join(test_path, i) for i in names_files]
len(files_to_score)

33

In [8]:
preds = []
for p in tqdm.tqdm_notebook(files_to_score):
    ds = SampleVideo(p, transform=transforms.Compose([ToTensor(),
                                Normalize([0.485, 0.456, 0.406],
                                          [0.229, 0.224, 0.225])]))
    dl = DataLoader(ds, batch_size=1, shuffle=False, drop_last=False)

    for sample in dl:
        images = sample['images']
        # full samples do not fit into GPU memory so evaluate sample in 'seq_length' batches
        batch = 0
        while batch * seq_length < images.shape[1]:
            if (batch + 1) * seq_length > images.shape[1]:
                image_batch = images[:, batch * seq_length:, :, :, :]
            else:
                image_batch = images[:, batch * seq_length:(batch + 1) * seq_length, :, :, :]
            #print(image_batch.shape)
            logits = model(image_batch.cuda())
            if batch == 0:
                probs = F.softmax(logits.data, dim=1).cpu().numpy()
            else:
                probs = np.append(probs, F.softmax(logits.data, dim=1).cpu().numpy(), 0)
            batch += 1

        events = np.argmax(probs, axis=0)[:-1]
        preds.append(events)
print('Predicted event frames: {}'.format(events))

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


  0%|          | 0/33 [00:00<?, ?it/s]

Predicted event frames: [195 450 513 267 270 487 238 572]


In [12]:
preds = np.sort(np.array(preds))

In [16]:
test_df = pd.read_csv('submission.csv', sep = ';')
len(set(test_df.iloc[:, 0].tolist()).intersection(set(os.listdir(test_path)))) / len(set(os.listdir(test_path)))

1.0

In [17]:
res_df = pd.DataFrame(np.array(preds))
res_df.columns = list(test_df.columns)[1:]

In [18]:
res_df[list(test_df.columns)[0]] = names_files

In [19]:
res_df = res_df[list(test_df.columns)]

In [20]:
res_df = pd.DataFrame(test_df.iloc[:, 0]).merge(res_df, left_on = 'Название видео', right_on = 'Название видео', how = 'left')

In [21]:
res_df.to_csv('subm.csv', index = False, sep = ';', encoding = 'utf-8')

In [22]:
res_df

,Название видео,P1,P2,P3,P4,P5,P7,P8,P10
0,f8b9e19d-f454-40f9-948d-907731823033.mp4,191,211,246,246,320,407,447,580
1,a73afde3-ea62-40bb-b6e4-4008a4d8ce0a.mp4,8,20,38,40,45,59,59,68
2,6a9de841-481e-4f0f-9b47-8ca2be41891b.mp4,0,0,1,40,40,40,40,50
3,f8f1e5c4-068d-405d-8953-d640aaa08517.mp4,14,29,39,48,60,63,64,67
4,1366a9b8-ec83-4f9a-9593-235ee4623269.mp4,25,29,31,58,164,187,188,568
5,4bd7b045-a997-40e5-a60d-1b05d28d0992.mp4,31,69,69,70,77,82,84,107
6,67ae2f8c-9e3e-4215-b31b-c8b38e74a09a.mp4,54,61,67,67,70,70,73,78
7,00858b25-c5c3-4092-8ae7-5c61ada25097.mp4,20,20,25,34,42,42,49,95
8,949e17f3-2818-4815-bb1c-f26a0c95d179.mp4,63,63,64,64,64,68,68,85
9,5badc618-782b-43f2-86e6-365279981524.mp4,195,238,267,270,450,487,513,572
